In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
from humanfriendly.terminal import output
from sympy.codegen.ast import float16
from torch.utils.tensorboard.summary import image

In [3]:
# ================================================================== #
#                     1. Basic autograd example 1                    #
# ================================================================== #

x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

# Build a computational graph.
y = w * x + b  #y = 2 * x + 3
y.backward()

# Print out the gradients.
print(x.grad)
print(w.grad)
print(b.grad)

tensor(2.)
tensor(1.)
tensor(1.)


In [13]:
# ================================================================== #
#                    2. Basic autograd example 2                     #
# ================================================================== #
# Create tensors of shape (10, 3) and (10, 2).
x = torch.randn(10,3)
y = torch.randn(10,2)

# print(x)
# print(y)
# Build a fully connected layer.
linear =   nn.Linear(3,2)
w = linear.weight
b = linear.bias

print ('w: ', w)
print ('b: ', b)

criterion = nn.MSELoss()
optimizer= torch.optim.SGD(linear.parameters(),lr = 0.01)
# Forward pass.
y_pred = linear(x)

# Compute the loss.
loss = criterion(y_pred, y)
print(loss.item())
loss.backward()

print("dL/dw:",  linear.weight.grad)
print("dL/db",linear.bias.grad)

optimizer.step()

pred = linear(x)
loss = criterion(pred, y)
print("After one optimizer step, loss:", loss.item())

w:  Parameter containing:
tensor([[ 0.3689, -0.5199,  0.0976],
        [ 0.2856,  0.5149, -0.0815]], requires_grad=True)
b:  Parameter containing:
tensor([ 0.0976, -0.3830], requires_grad=True)
1.3297983407974243
dL/dw: tensor([[ 0.6949, -0.6964, -0.3265],
        [-0.2497, -0.0506, -0.0406]])
dL/db tensor([ 0.4565, -0.2645])
After optimizer step, loss: 1.3157014846801758


In [21]:
# ================================================================== #
#                     3. Loading data from numpy                     #
# ================================================================== #
x = np.array([[1.,2.],[3.,4.]],dtype=np.float32)
y = torch.from_numpy(x)
z = y.numpy()
z

array([[1., 2.],
       [3., 4.]], dtype=float32)

In [37]:
# ================================================================== #
#                         4. Input pipeline                           #
# ================================================================== #

train_dataset = torchvision.datasets.CIFAR10(
    root='../cifar10',
    train=True,
    download=True,
    transform=transforms.ToTensor()
)

image,label = train_dataset[0]
print (image.size())
print (label)

# data loader
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=64,
    shuffle=True,
)

print(len(train_loader))
dataiter = iter(train_loader)

images, labels = next(dataiter)
print(images.size())
print(labels.size())



torch.Size([3, 32, 32])
6
782
torch.Size([64, 3, 32, 32])
torch.Size([64])


In [ ]:
# ================================================================== #
#                5. Input pipeline for custom dataset                 #
# ================================================================== #
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        # TODO
        pass
    def __getitem__(self, index):
        # TODO
        pass
    def __len__(self):
        return 0

# usage
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(
    dataset=custom_dataset,
    batch_size=64,
    shuffle=True,
)

In [ ]:
# ================================================================== #
#                        6. Pretrained model                         #
# ================================================================== #
resnet = torchvision.models.resnet18(pretrained=True)
for param in resnet.parameters():
    param.requires_grad = False  # Freeze the parameters
nn.Linear(resnet.fc.in_features, 100)
images = torch.randn(64,3,32,32)
output = resnet(images)
print(output.size())

In [ ]:
# ================================================================== #
#                      7. Save and load the model                    #
# ================================================================== #

torch.save(resnet, 'resnet.pth')  # Save the entire model
model = torch.load('resnet.pth')  # Load the entire model

torch.save(resnet.state_dict(), 'resnet.pth')# Save and load only the model parameters
model = torch.load('resnet.pth')